In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir /content/lightning_logs

# %tensorboard --logdir /content/drive/MyDrive/CVProject/lightning_logs

In [ ]:
#Install required libraries
!pip install datasets
!pip install torchmetrics
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
"""
This snippet of code downloads data from pyronear and creates the appropriate directory structure.

"""
import os
from datasets import load_dataset

# Define paths
REPO_ID = "pyronear/pyro-sdis"
OUTPUT_DIR = "./pyro-sdis"
IMAGE_DIR = os.path.join(OUTPUT_DIR, "images")
LABEL_DIR = IMAGE_DIR.replace("images", "labels")

# Create the directory structure
for split in ["train", "val"]:
    os.makedirs(os.path.join(IMAGE_DIR, split), exist_ok=True)
    os.makedirs(os.path.join(LABEL_DIR, split), exist_ok=True)

# Load the dataset from the Hugging Face Hub
dataset = load_dataset(REPO_ID)

# Save images and labels to disk
for split in ["train", "val"]:
    split_data = dataset[split]
    # split_data = split_data.select(range(1000))
    for example in split_data:
        # Extract fields
        image = example["image"]
        annotations = example["annotations"]
        image_name = example["image_name"]

        # Save image
        image_path = os.path.join(IMAGE_DIR, split, image_name)
        image.save(image_path)

        # Save annotations (labels)
        label_path = os.path.join(LABEL_DIR, split, os.path.splitext(image_name)[0] + ".txt")
        with open(label_path, "w") as f:
            f.write(annotations)

print(f"Dataset saved to {OUTPUT_DIR}")


README.md:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/485M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

...

train-00005-of-00006.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/390M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29537 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/4099 [00:00<?, ? examples/s]

Dataset saved to ./pyro-sdis


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
"""
Dataset to load smoke data from pyronear.
Transforms should be from albumentation library rather than Torchvision Library.
"""

import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from torchvision.transforms import ToTensor

class PyroSDISDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, class_offset=0):
        self.images_dir = os.path.join(root_dir, 'images', split)
        self.labels_dir = os.path.join(root_dir, 'labels', split)
        self.image_filenames = sorted([f for f in os.listdir(self.images_dir) if f.endswith(('.jpg', '.png'))])
        self.transform = transform
        self.class_offset = class_offset

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        image_path = os.path.join(self.images_dir, image_filename)
        label_path = os.path.join(self.labels_dir, os.path.splitext(image_filename)[0] + '.txt')

        image = np.array(Image.open(image_path).convert("RGB"))
        H, W, _ = image.shape

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    cls, x_c, y_c, w, h = map(float, line.strip().split())

                    x_c *= W
                    y_c *= H
                    w *= W
                    h *= H
                    # print("H",H)
                    # print("W",W)
                    # print("x_c: ",x_c)
                    # print("y_c: ",y_c)
                    # print("w:",w)
                    # print("h:",h)

                    # x_min = x_c - w / 2
                    # y_min = y_c - h / 2
                    # x_max = x_c + w / 2
                    # y_max = y_c + h / 2
                    x_min = max(int(x_c - w / 2), 0)
                    y_min = max(int(y_c - h / 2), 0)
                    #-1 because image indexing is zero based. the last pixel would be W - 1
                    x_max = min(int(x_c + w / 2), W - 1)
                    y_max = min(int(y_c + h / 2), H - 1)


                    boxes.append([x_min, y_min, x_max, y_max])
                    labels.append(int(cls) + self.class_offset)


        if len(boxes) == 0:
            boxes = np.empty((0,4))

        boxes = np.array(boxes)
        labels = np.array(labels, dtype=np.int64)



        if self.transform:
            transformed = self.transform(
                image=image,
                bboxes=boxes,
                class_labels=labels
            )
            image = transformed['image']


            boxes = torch.tensor(transformed['bboxes'], dtype=torch.float32)
            labels = torch.tensor(transformed['class_labels'], dtype=torch.int64)
        else:
            image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)

        #Convert image to between 0-1
        image = image/255.0


        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx])
        }


        return image, target


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

#Albumentations transform to be passed into dataset
transform = A.Compose([
    A.HorizontalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # simulates when the camera is moving quickly.
    A.MotionBlur(p=0.1),
    # basic blurring
    A.Blur(blur_limit=3, p=0.1),
    A.ColorJitter(p=0.1),
    A.RandomFog(p=0.1),

    ToTensorV2(p=1.0)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'], min_visibility=0.1,
                            filter_invalid_bboxes=True
                            ))

In [ ]:
def collate_fn(batch):
    # Filter out None values (images with no bounding boxes)
    # print("batch before: 1",batch)

    batch = [item for item in batch if item is not None]

    # print("batch after: ",batch)
    # If the batch is empty, return None
    if len(batch) == 0:
        return None, None

    # if len(batch) == 0:
    #     return [], []  # Return empty lists for images and targets

    # Collate images and targets
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]

    return images, targets

In [ ]:
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

batch_size = 12


train_dataset = PyroSDISDataset('pyro-sdis', split='train', transform=transform)
val_dataset = PyroSDISDataset('pyro-sdis', split='val', transform=transform)

# subset_indices = list(range(300))
# subset_indices_val = list(range(100))
# from torch.utils.data import Subset

# train_dataset = Subset(train_dataset, subset_indices)
# val_dataset = Subset(val_dataset, subset_indices_val)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers = 2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers = 2)



In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_V2_Weights

from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.ops import box_iou

In [ ]:
#Pytorch Lightning implementation

from pytorch_lightning import LightningModule
# from pytorch_lightning.core.decorators import auto_move_data
from torchmetrics.detection.mean_ap import MeanAveragePrecision

class FasterRCNNModel(LightningModule):
    def __init__(
        self,
        num_classes=2,
        batch_size=8,
    ):
        super().__init__()

        self.map_metric = MeanAveragePrecision()
        self.batch_size = batch_size
        # Number of classes (including background).
        self.num_classes = num_classes

        # Load the model with pre-trained weights
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        model.to(device)

        self.model = model

        #model.parameters have been change to params


    def forward(self, images, targets):
        return self.model(images, targets)

    def configure_optimizers(self):
        # Set up the Adam optimizer

        params = [p for p in self.model.parameters() if p.requires_grad]

        learning_rate=0.001
        beta1 = 0.9            # Default value for beta1
        beta2 = 0.999          # Default value for beta2
        epsilon = 1e-8         # Default value for epsilon
        weight_decay = 0.0001  # Optional L2 regularization
        optimizer = torch.optim.Adam(
            params,
            lr=learning_rate,
            betas=(beta1, beta2),
            eps=epsilon,
            weight_decay=weight_decay
        )

        return optimizer


    def training_step(self, batch, batch_idx):

        images, targets = batch

        # if images is None or targets is None:
        #     continue

        # Forward pass
        loss_dict =self.model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        self.log("train_loss", losses, on_step=True, on_epoch=True, prog_bar=True,
                 logger=True, batch_size=batch_size)

        return losses


    @torch.no_grad()
    def validation_step(self, batch, batch_idx):


        images, targets = batch

        # if images is None or targets is None:
        #         continue



        predictions = self.model(images)


        #KJ: Model is switched to training mode in trainingstep to force a dictionary to be returned. In eval mode, a list of predictions is returned instead.
        self.model.train()
        loss_dict = self.model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        self.log("valid_loss", losses, on_step=True, on_epoch=True, prog_bar=True,
                 logger=True, batch_size=batch_size)

        self.model.eval()
        for pred, target in zip(predictions, targets):
            if pred["scores"].nelement() == 0:
                continue

            # Update mAP metric
            self.map_metric.update([pred], [target])

            # Compute IoU between predicted and ground truth boxes
            pred_boxes = pred['boxes']
            gt_boxes = target['boxes']
            if pred_boxes.numel() == 0 or gt_boxes.numel() == 0:
                continue

        return {'loss':losses, 'batch_predictions':predictions}

    def on_validation_epoch_end(self):

        #KJ added
        #self.map_metric compute returns a map_dictionary
        #https://lightning.ai/docs/torchmetrics/stable/detection/mean_average_precision.html
        #types of map keys available in documentation above.
        map_metrics = self.map_metric.compute()

        self.log("mAP", map_metrics["map"], prog_bar=True, logger=True,batch_size=batch_size)
        self.log("mAP_50", map_metrics["map_50"], prog_bar=True, logger=True, batch_size=batch_size)
        self.map_metric.reset()


    # def predict_step(self, batch, batch_idx, dataloader_idx=0):
    #     images, _, _, image_ids = batch

    #     # Forward pass through the model to get raw predictions
    #     outputs = self.model(images, None)
    #     detections = outputs["detections"]

    #     # Prepare the predictions in the desired format
    #     predictions = []
    #     for i in range(detections.shape[0]):
    #         pred_bboxes = detections[i][:, :4]
    #         pred_scores = detections[i][:, -2]
    #         pred_labels = detections[i][:, -1].int()
    #         predictions.append({
    #             "image_id": image_ids[i],
    #             "boxes": pred_bboxes,
    #             "scores": pred_scores,
    #             "labels": pred_labels
    #         })

    #     return predictions

In [ ]:
#Code here is to train the model from scratch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

#Set up model checkpoint for model and lightning logs to be saved.
LOG_DIR = "/content/drive/MyDrive/CVProject/FasterRCNN/lightning_logs/"

logger = TensorBoardLogger(
    save_dir=LOG_DIR,
    name = "faster_rcnn_with_aug",
    version = "run_03"
)

checkpoint_callback = ModelCheckpoint(
    dirpath='/content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_03',
    filename='best-checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='valid_loss',
    mode='min'
)

trainer = Trainer(accelerator='gpu', devices=[0], max_epochs=30, num_sanity_val_steps=1, logger=logger, callbacks = [checkpoint_callback])
fasterRCNNModel = FasterRCNNModel(batch_size=batch_size)
trainer.fit(fasterRCNNModel,train_loader,val_loader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:00<00:00, 224MB/s]
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 2462: 'valid_loss' reached 0.14633 (best 0.14633), saving model to '/content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_03/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 10, global step 27082: 'valid_loss' reached 0.12514 (best 0.12514), saving model to '/content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_03/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 11, global step 29544: 'valid_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 12, global step 32006: 'valid_loss' was not in top 1


In [ ]:
# #Code here is to load from existing checkpoints
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks import ModelCheckpoint

# #Resume training without load
# ckpt_path = "/content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_02/best-checkpoint.ckpt"

# fasterRCNNModel = FasterRCNNModel()

# trainer = Trainer(accelerator='gpu', devices=[0], max_epochs=20, num_sanity_val_steps=1)
# trainer.fit(fasterRCNNModel,train_loader,val_loader, ckpt_path=ckpt_path)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# Path to checkpoint to resume from
ckpt_path = "/content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_02/best-checkpoint.ckpt"

# Logger
logger = TensorBoardLogger(
    save_dir="/content/drive/MyDrive/CVProject/FasterRCNN/lightning_logs/",
    name="faster_rcnn_with_aug",
    version="run_02"  # Same version to resume logging
)

# Checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_02",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="valid_loss",
    mode="min",
    verbose=True
)

# Model
fasterRCNNModel = FasterRCNNModel(batch_size = batch_size)

# Trainer with logger and callback
trainer = Trainer(
    accelerator='gpu',
    devices=[0],
    max_epochs=20,
    num_sanity_val_steps=1,
    logger=logger,
    callbacks=[checkpoint_callback]
)

# Resume training
trainer.fit(fasterRCNNModel, train_loader, val_loader, ckpt_path=ckpt_path)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_02 exists and is not empty.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/CVProject/FasterRCNN/checkpoints/run_02/best-chec

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 19696: 'valid_loss' was not in top 1


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 12, global step 32006: 'valid_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 13, global step 34468: 'valid_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
#V4
import os
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchmetrics import Precision, Recall, F1Score

# Define checkpoint directory

# checkpoint_dir = "./checkpoints"
checkpoint_dir = "/content/drive/MyDrive/CVProject/manual_checkpoints/Finetuned"

os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize metrics
# precision_metric = Precision(task="binary", average="macro")
# recall_metric = Recall(task="binary", average="macro")
# f1_metric = F1Score(task="binary", average="macro")
map_metric = MeanAveragePrecision(iou_type="bbox")